In [ ]:
import os
import openai
import gradio as gr
from dotenv import load_dotenv  # 🔄 추가된 부분

# 🔐 OpenAI API 키 가져오기
load_dotenv()  # .env 파일에서 환경변수 읽기

api_key = os.getenv("OPENAI_API_KEY")  # 변수명 수정
if api_key is None:
    raise ValueError("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

# Gradio interface 함수
def respond(user_message, chat_history):
    # 기존 채팅 기록에 사용자 메시지 추가
    chat_history.append((user_message, ""))
    messages = []
    
    # 채팅 기록을 OpenAI API 형식으로 변환
    for user, assistant in chat_history:
        messages.append({"role": "user", "content": user})
        if assistant:  # 빈 응답이 아닌 경우만 추가
            messages.append({"role": "assistant", "content": assistant})
    
    # 응답 생성 및 스트리밍
    bot_response = ""
    for chunk in openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
        stream=True,
    ):
        if chunk.choices[0].delta.content:
            bot_response += chunk.choices[0].delta.content
            chat_history[-1] = (user_message, bot_response)
            yield chat_history

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="메시지를 입력하세요...", show_label=False, lines=1)
    clear = gr.Button("대화 초기화")
    
    msg.submit(respond, [msg, chatbot], chatbot)
    msg.submit(lambda: "", None, msg, queue=False)  # 입력 필드 비우기
    clear.click(lambda: [], None, chatbot, queue=False)  # 대화 초기화

if __name__ == "__main__":
    demo.launch()

In [ ]:
### 멀티 턴 구현

import os
import openai
import gradio as gr
from dotenv import load_dotenv  # 🔄 추가된 부분

# 🔐 OpenAI API 키 가져오기
load_dotenv()  # .env 파일에서 환경변수 읽기

api_key = os.getenv("OPENAI_API_KEY")  # 변수명 수정
if api_key is None:
    raise ValueError("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

# 멀티턴 대화 처리 함수
def chat(message, history):
    """사용자 메시지를 받아 AI 응답을 생성하는 함수"""
    # 대화 기록 초기화
    history = history or []
    
    # OpenAI API에 전송할 메시지 형식으로 변환
    messages = [{"role": "system", "content": "저는 도움을 주는 AI 어시스턴트입니다. 어떻게 도와드릴까요?"}]
    
    # 대화 기록 추가
    for human, ai in history:
        messages.append({"role": "user", "content": human})
        if ai:  # AI 응답이 있는 경우에만 추가
            messages.append({"role": "assistant", "content": ai})
    
    # 새 메시지 추가
    messages.append({"role": "user", "content": message})
    
    # 새 사용자 메시지를 기록에 추가
    history.append((message, ""))
    
    # 응답 생성 및 스트리밍
    full_response = ""
    for chunk in openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
        stream=True,
    ):
        if chunk.choices[0].delta.content is not None:
            full_response += chunk.choices[0].delta.content
            # 마지막 대화 항목의 AI 응답 부분만 업데이트
            history[-1] = (message, full_response)
            yield history

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    gr.Markdown("# 멀티턴 대화 AI 챗봇")
    
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=(None, "https://api.dicebear.com/7.x/bottts/svg?seed=gpt"),
        height=600
    )
    
    with gr.Row():
        msg = gr.Textbox(
            placeholder="메시지를 입력하세요...", 
            show_label=False,
            lines=1,
            container=False,
            scale=12
        )
        submit_btn = gr.Button("전송", scale=1)
    
    with gr.Row():
        clear_btn = gr.Button("대화 초기화")
        
    # 이벤트 핸들러
    msg.submit(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=chatbot,
        queue=True
    ).then(
        fn=lambda: "",
        inputs=None,
        outputs=msg
    )
    
    submit_btn.click(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=chatbot,
        queue=True
    ).then(
        fn=lambda: "",
        inputs=None,
        outputs=msg
    )
    
    clear_btn.click(lambda: [], None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(share=True)  # share=True로 설정하면 공개 URL이 생성됩니다, False